In [1]:
import torch

### create tensor data

In [2]:
x = torch.randn(5, 3)
y = torch.ones_like(x)
# tensor = torch.ones(5, 3)

In [3]:
x,y

(tensor([[-0.3788,  0.2000, -0.0808],
         [-1.3474, -0.0264, -0.5016],
         [ 0.7819,  2.3440, -0.4746],
         [-1.6287,  0.0484,  0.6700],
         [ 1.6000, -0.5406,  0.2774]]),
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]))

In [4]:
z1 = x + y
z1

tensor([[ 0.6212,  1.2000,  0.9192],
        [-0.3474,  0.9736,  0.4984],
        [ 1.7819,  3.3440,  0.5254],
        [-0.6287,  1.0484,  1.6700],
        [ 2.6000,  0.4594,  1.2774]])

### Moving to GPU

In [5]:
x = x.cuda()
y = y.cuda()
z2 = x + y
x,y,z2

(tensor([[-0.3788,  0.2000, -0.0808],
         [-1.3474, -0.0264, -0.5016],
         [ 0.7819,  2.3440, -0.4746],
         [-1.6287,  0.0484,  0.6700],
         [ 1.6000, -0.5406,  0.2774]], device='cuda:0'),
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]], device='cuda:0'),
 tensor([[ 0.6212,  1.2000,  0.9192],
         [-0.3474,  0.9736,  0.4984],
         [ 1.7819,  3.3440,  0.5254],
         [-0.6287,  1.0484,  1.6700],
         [ 2.6000,  0.4594,  1.2774]], device='cuda:0'))

In [6]:
### 数据必须在同一设备才能操作

In [7]:
try: z1 + z2
except Exception as e: print(e)

Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!


In [8]:
z2 = z2.to("cpu")
print(z1 + z2)

tensor([[ 1.2424,  2.3999,  1.8385],
        [-0.6948,  1.9472,  0.9968],
        [ 3.5639,  6.6881,  1.0507],
        [-1.2574,  2.0968,  3.3399],
        [ 5.2001,  0.9188,  2.5548]])


### Neural network

In [9]:
import torch.nn as nn
import torch.nn.functional as F

# 定义一个简单的神经网络类
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # super(Net, self).__init__()
        # 定义每一层卷积神经网络，输入通道维度=1，输出通道维度=6，卷积核大小3*3
        self.conv1 = nn.Conv2d(1, 6, 3)
        # 定义第二层卷积神经网络，输入通道维度=6，输出通道维度=16，卷积核大小3*3
        self.conv2 = nn.Conv2d(6, 16, 3)   #6×6×16
        # 定义三层全连接网络
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 在(2,2)的池化窗口下执行最大池化操作
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) #
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        # 计算size,除了第0个 维度上的batch_size
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

1. 第一层卷积：
   - 输入大小：32×32
   - 卷积核大小：3×3
   - 输出通道维度：6
   - 使用零填充（padding=0），步幅（stride=1）
   - 计算公式：$output\_size = \frac{{input\_size - kernel\_size + 2 \times padding}}{{stride}} + 1 = \frac{{32 - 3 + 2 \times 0}}{{1}} + 1 = 30$
   - 输出大小：30×30

2. 第一次最大池化：
   - 输入大小：30×30
   - 池化窗口大小：2×2
   - 使用步幅（stride=2）
   - 计算公式：$output\_size = \frac{{input\_size}}{{stride}} = \frac{{30}}{{2}} = 15$
   - 输出大小：15×15

3. 第二层卷积：
   - 输入大小：15×15
   - 卷积核大小：3×3
   - 输出通道维度：16
   - 使用零填充（padding=0），步幅（stride=1）
   - 计算公式：$output\_size = \frac{{input\_size - kernel\_size + 2 \times padding}}{{stride}} + 1 = \frac{{15 - 3 + 2 \times 0}}{{1}} + 1 = 13$
   - 输出大小：13×13

4. 第二次最大池化：
   - 输入大小：13×13
   - 池化窗口大小：2×2
   - 使用步幅（stride=2）
   - 计算公式：$output\_size = \frac{{input\_size}}{{stride}} = \frac{{13}}{{2}} = 6.5$（结果向下取整）
   - 输出大小：6×6

最后，在卷积和池化的过程中，图像的大小从32×32逐步减小到6×6。在这之后，将数据展平，并通过全连接层进行处理。

![16970320504401697032050389.png](https://fastly.jsdelivr.net/gh/Chenjiangwen/ImageHostingService@main/pic/16970320504401697032050389.png)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('img.png')

fig = plt.figure(figsize=(20, 15))
plt.imshow(img)
plt.axis('on')
plt.show()


In [ ]:
net = Net()
net.to("cuda")

In [ ]:
for name, param in net.named_parameters():
    print('{: <30} {: <30} {: <20}'.format(name, str(param.shape), str(param.numel())))
print("总参数量： {}".format(sum(x.numel() for x in net.parameters())))

### 输入、输出

In [ ]:
input = torch.randn(1, 1, 32, 32).to("cuda")
print('input', input.size(), input.type())
out = net(input)
print(out)


### 损失

In [ ]:
target = torch.randn(10).cuda()
print(target.shape)
target = target.view(1,-1)
print(target.size())
criterion = nn.MSELoss()
loss = criterion(out, target)
print(loss)

### Input -> Conv1 -> ReLu -> MaxPool -> Conv2 -> ReLu -> MaxPool
###      -> view -> FC_linear -> Relu -> FC_linear -> ReLu -> FC_linear
###      -> MSELoss
###      ->loss

In [ ]:
net.zero_grad()  #梯度清零，否则不同批次数据之间的梯度会累加
print(net.conv1.bias.grad)

loss.backward()

print(net.conv1.bias.grad)

In [ ]:
### 反向传播 loss.backward()

In [ ]:
# 跟踪loss反向传播
print(loss.grad_fn)  #MSELoss
print(loss.grad_fn.next_functions[0][0])  #linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])   #ReLu

### 优化器 更新网络参数

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()

output = net(input)
loss = criterion(output, target)

loss.backward()

# 更新参数
optimizer.step()

print('总结：1.构建网络 2.定义损失函数 3.反向传播 4.更新参数\n'
      '循环训练直到满意loss，结束训练 -> 模型 -> do something')

## cifar-10

| 类别  | plane | car | bird | cat | deer | dog | frog | horse | ship | truck |
| ----- | ----- | --- | ---- | --- | ---- | --- | ---- | ----- | ---- | ----- |
| 准确率 | 69%   | 69% | 27%  | 31% | 42%  | 64% | 59%  | 65%   | 59%  | 62%   |

https://cjwen-imageclassification-resnet18-cifar10.streamlit.app/